# 자연어 인식

In [32]:
from sklearn.model_selection import train_test_split
import re
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
import json
import nltk
import pyaudio
from konlpy.tag import *
kkma=Kkma()
tw=Twitter()
po = pyaudio.PyAudio()
for index in range(po.get_device_count()): 
    desc = po.get_device_info_by_index(index)
    #if desc["name"] == "record":
    print ("DEVICE: %s  INDEX:  %s  RATE:  %s " %  (desc["name"], index,  int(desc["defaultSampleRate"])))


DEVICE: Microsoft »ç¿îµå ¸ÅÆÛ - Input  INDEX:  0  RATE:  44100 
DEVICE: ¸¶ÀÌÅ© ¹è¿­(Realtek High Defini  INDEX:  1  RATE:  44100 
DEVICE: Microsoft »ç¿îµå ¸ÅÆÛ - Output  INDEX:  2  RATE:  44100 
DEVICE: ½ºÇÇÄ¿(Realtek High Definition   INDEX:  3  RATE:  44100 


In [5]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("Start to record the audio.")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Recording is finished.")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

Start to record the audio.
Recording is finished.


In [6]:
import urllib3
import json
import base64
openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
accessKey = ""
audioFilePath = "output.wav"
languageCode = "korean"
 
file = open(audioFilePath, "rb")
audioContents = base64.b64encode(file.read()).decode("utf8")
file.close()
 
requestJson = {
    "access_key": accessKey,
    "argument": {
        "language_code": languageCode,
        "audio": audioContents
    }
}
 
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
)
 
print("[responseCode] " + str(response.status))
print("[responBody]")

[responseCode] 200
[responBody]


In [52]:
jsonData=json.loads(response.data.decode('utf-8'))
print(jsonData['return_object']['recognized'])

왼쪽으로 돌아


In [53]:
resultT=tw.pos(jsonData['return_object']['recognized'])

In [54]:
import csv
Command=[]
Answer=[]
f = open('command.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
next(rdr)
for line in rdr:
    Command.append(line[0])
    Answer.append(int(line[1]))
f.close()

In [55]:
x_text_list=[]
y_text_list=[]
for nowC,nowA in zip(Command,Answer):
    tmp=''
    arr=tw.pos(nowC)
    for pos in arr:
        if pos[1] == 'Noun' or pos[1] == 'Verb':
            tmp=tmp+pos[0]+' '
    x_text_list.append(tmp+tmp+tmp)
    y_text_list.append(nowA)
tmp=''
result=[]
for pos in resultT:
    if pos[1] == 'Noun' or pos[1] == 'Verb':
            tmp=tmp+pos[0]+' '
result.append(tmp)

In [56]:
gsvc=joblib.load('model.pkl')
vectorizer = CountVectorizer()
vectorizer.fit_transform(x_text_list)

<19x13 sparse matrix of type '<class 'numpy.int64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [57]:
Result = gsvc.predict(vectorizer.transform(result))

In [58]:
print(Result)

[1]
